# Purpose
This notebook is an experiment of stepwise inferencing, we load the models, and do the inference on daily basis. 
Gracefully phasing into the market, only do the operations when the value of it is bigger than a threshold.

In [1]:
import sys
import os
import numpy as np
sys.path.append('..')
from util import load_pkl
from util import create_if_not_exist, generate_features, get_X_y_by_stock, get_tickers, get_currency_pair, load_latest_price_data, convert, add_features, load_pkl, merge_fred, remove_nan
import logging
logger = logging.getLogger('training')

In [4]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

BASE_LINE_HORIZON = 256

# Your function to load pickle files

def load_pkl(file_path):
    with open(file_path, 'rb') as file:
        return pd.read_pickle(file)

# Predict for each period independently
def predict_for_period(period):
    data_dir = f'../processed_data_{period}'
    S = load_pkl(f'{data_dir}/S.pkl')

    df_train_X_all = load_pkl(f"{data_dir}/df_train_X_all.pkl")
    df_train_y_all = load_pkl(f"{data_dir}/df_train_y_all.pkl")
    df_test_X_all = load_pkl(f"{data_dir}/df_test_X_all.pkl")
    df_test_y_all = load_pkl(f"{data_dir}/df_test_y_all.pkl")

    with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
        valid_tickers = [line.strip() for line in f.readlines()]

    all_y_pred = None

    for i, stock_name in enumerate(valid_tickers):
        df_train_X = df_train_X_all[i]
        df_train_y = df_train_y_all[i]
        df_test_X = df_test_X_all[i]
        df_test_y = df_test_y_all[i]

        X_train = df_train_X.values
        y_train = df_train_y.values.ravel()
        X_test = df_test_X.values

        try:
            rf_model_path = f'{data_dir}/models/{stock_name}_rf.pkl'
            svr_model_path = f'{data_dir}/models/{stock_name}_svr.pkl'

            best_pipeline_rf = load_pkl(rf_model_path)
            best_pipeline_svr = load_pkl(svr_model_path)

            y_pred_rf = best_pipeline_rf.predict(X_test)
            y_pred_svr = best_pipeline_svr.predict(X_test)

            df_test_X_naive = pd.concat((df_train_X[-512:], df_test_X))
            divisor = 512 / period
            y_pred_naive = (df_test_X_naive[f'log_price_diff_512']
                            .rolling(window=512).mean()[512:] / divisor).to_numpy()

            y_pred = (y_pred_rf + y_pred_svr + y_pred_naive) / 3

            y_pred = y_pred * (BASE_LINE_HORIZON / period)

            df_y_pred = pd.DataFrame(y_pred, index=df_test_y.index, columns=[stock_name])
            if all_y_pred is None:
                all_y_pred = df_y_pred
            else:
                all_y_pred = pd.concat((all_y_pred, df_y_pred), axis=1)

        except Exception as e:
            logging.error(f'Error in predicting {stock_name}: {e}')
            continue

    return period, all_y_pred.fillna(method='ffill').fillna(method='bfill')

# Parallel execution of multiple periods
periods = [8, 16, 32, 64, 128, 256]
all_y_pred_all_periods = {}

with ThreadPoolExecutor(max_workers=len(periods)) as executor:
    future_to_period = {executor.submit(predict_for_period, period): period for period in periods}
    for future in as_completed(future_to_period):
        period, result = future.result()
        all_y_pred_all_periods[period] = result
        print(f'Completed predictions for period: {period}')

Completed predictions for period: 128
Completed predictions for period: 256
Completed predictions for period: 8
Completed predictions for period: 16
Completed predictions for period: 64
Completed predictions for period: 32


In [25]:
all_y_pred_all_periods.items()

dict_items([(128,               ADS.DE    AIR.DE    ALV.DE    BAS.DE   BAYN.DE    BEI.DE  \
Date                                                                     
2019-01-02  0.280136  0.152933  0.068366  0.076757 -0.148454  0.113791   
2019-01-03  0.277564  0.139380  0.059641  0.069186 -0.154757  0.112857   
2019-01-04  0.291759  0.160028  0.074122  0.101527 -0.069236  0.114560   
2019-01-07  0.337692  0.202279  0.139781  0.155654 -0.005511  0.125038   
2019-01-08  0.365100  0.216768  0.141636  0.158683  0.009436  0.123648   
...              ...       ...       ...       ...       ...       ...   
2024-05-21  0.265802  0.469399  0.260178  0.251699 -0.057002  0.225066   
2024-05-22  0.254796  0.445664  0.240670  0.253574 -0.053288  0.224035   
2024-05-23  0.222926  0.454416  0.208580  0.243482 -0.090370  0.216876   
2024-05-24  0.213122  0.411948  0.192594  0.236575 -0.003900  0.221870   
2024-05-27  0.179509  0.388652  0.191049  0.253805  0.001050  0.209661   

              BMW.D

In [33]:
import pandas as pd
from util import do_optimization, get_errors_mu_short, get_shrinkage_covariance, save_json_to_dir, update_stock_operation_and_weight
# Join all DataFrames in all_y_pred_all_periods into a single wider DataFrame

def join_all_period_predictions(all_y_pred_all_periods):
    wider_df = None
    all_errors = None
    for period in periods:
        data_dir = f'../processed_data_{period}'
        df_err = pd.read_csv(os.path.join(data_dir, 'all_errors.csv'),  index_col=0, parse_dates=True)
        avg_err = df_err.mean(axis=1).mean()
        df_err = df_err * np.sqrt(BASE_LINE_HORIZON / period)
        print(f'Average error for time horizon {period}: {avg_err}')

        df = all_y_pred_all_periods[period]
        df_renamed = df.add_suffix(f'_{period}')

        if wider_df is None:
            wider_df = df_renamed
        else:
            wider_df = wider_df.join(df_renamed, how='outer')

        if all_errors is None:
            all_errors = df_err
        else:
            all_errors = pd.concat([all_errors, df], axis=1, join='outer')

    return wider_df, all_errors

# Execute the join
wider_table, all_errors = join_all_period_predictions(all_y_pred_all_periods)

final_tickers = list(all_errors.columns)
all_errors = all_errors.fillna(method='ffill').fillna(method='bfill')
S = get_shrinkage_covariance(all_errors)
#sorted_columns = sorted(wider_table.columns, key=custom_sort)
#wider_table = wider_table[sorted_columns]
print(wider_table.head())


Average error for time horizon 8: 0.0010003566851808808
Average error for time horizon 16: 0.0020573815107795056
Average error for time horizon 32: 0.0025075842707670366
Average error for time horizon 64: 0.006813559244519105
Average error for time horizon 128: 0.033682289355474904
Average error for time horizon 256: 0.02836063579119684
            ADS.DE_8  AIR.DE_8  ALV.DE_8  BAS.DE_8  BAYN.DE_8  BEI.DE_8  \
Date                                                                      
2019-01-02  0.254858  0.109586  0.257197  0.028904  -0.567610  0.115057   
2019-01-03  0.248780  0.145758  0.252820 -0.007147  -0.586287  0.111038   
2019-01-04  0.278827  0.094631  0.260074  0.092268  -0.287270  0.115216   
2019-01-07  0.252133  0.063723  0.253737  0.031591  -0.373124  0.104516   
2019-01-08  0.253248  0.044006  0.253707  0.035627  -0.297597  0.102574   

            BMW.DE_8  BNR.DE_8  CBK.DE_8  CON.DE_8  ...  IUSG_256   VGK_256  \
Date                                                ... 

In [34]:
S.shape

(4476, 4476)

In [27]:
wider_table.shape

(1518, 4476)

In [23]:
# get the price data
all_df_test_X = None
for period in periods:

    data_dir = f'../processed_data_{period}'
    df_test_X_all = load_pkl(f"{data_dir}/df_test_X_all.pkl")
    with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
        valid_tickers = [line.strip() for line in f.readlines()]

    # merge

    for i, stock_name in enumerate(valid_tickers):
        df_test_X = df_test_X_all[i][['log_price_diff_1']].shift(-1)
        df_test_X = df_test_X.rename(columns={'log_price_diff_1': f'{valid_tickers[i]}'})
        if all_df_test_X is None:
            all_df_test_X = df_test_X
        elif valid_tickers[i] not in all_df_test_X.columns:
            all_df_test_X = pd.concat((all_df_test_X, df_test_X), axis=1)

all_df_test_X.head()

,ADS.DE,AIR.DE,ALV.DE,BAS.DE,BAYN.DE,BEI.DE,BMW.DE,BNR.DE,CBK.DE,CON.DE,...,IUSG,VGK,EFV,XLI,SHV,XLY,ACWI,IUSV,IYW,VHT
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,0.001248,-0.032606,-0.006878,-0.007904,0.002705,-0.001767,-0.006252,-0.039846,0.010089,0.010303,...,-0.025430,-0.007474,-0.004669,-0.030902,0.000363,-0.021890,-0.017140,-0.022763,-0.047400,-0.022852
2019-01-03,0.038836,0.048956,0.024930,0.052433,0.066222,0.009900,0.038764,0.053349,0.058075,0.038156,...,0.033954,0.029565,0.032020,0.037215,0.000271,0.032558,0.031382,0.030640,0.044463,0.031335
2019-01-04,-0.002021,-0.013420,-0.003082,0.000471,-0.010927,-0.004741,0.010772,0.001177,0.011399,0.008240,...,0.008451,0.004441,0.001941,0.008102,0.000000,0.022360,0.006129,0.007216,0.008412,0.008846
2019-01-07,0.021865,0.035127,-0.001072,0.002979,0.023411,-0.008690,-0.000848,0.021042,0.021563,0.013214,...,0.011713,0.008024,0.005799,0.013610,-0.000090,0.010996,0.007306,0.008353,0.011046,0.010673
2019-01-08,0.004648,0.041071,0.012855,0.008705,0.008034,0.002324,0.017162,0.027591,-0.010564,0.037277,...,0.004611,0.012509,0.008743,0.006736,-0.000091,0.005071,0.009208,0.004742,0.010197,0.003920


In [ ]:
# Merge the matrix S.


In [10]:
df_test_X_all[0]['log_price_diff_1'].shift(-1)

Date
2019-01-02    0.001248
2019-01-03    0.038836
2019-01-04   -0.002021
2019-01-07    0.021865
2019-01-08    0.004647
                ...   
2024-08-19   -0.012969
2024-08-20    0.018622
2024-08-21   -0.002146
2024-08-22    0.013349
2024-08-23         NaN
Name: log_price_diff_1, Length: 1438, dtype: float64

In [5]:
all_y_pred_all_periods[8]

,ADS.DE,AIR.DE,ALV.DE,BAS.DE,BAYN.DE,BEI.DE,BMW.DE,BNR.DE,CBK.DE,CON.DE,...,IUSG,VGK,EFV,XLI,SHV,XLY,ACWI,IUSV,IYW,VHT
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,0.254858,0.109586,0.257197,0.028904,-0.567610,0.115057,0.185363,-0.149725,-0.228552,-0.721357,...,0.277914,0.134299,0.031670,0.226328,0.005112,0.285334,0.095154,0.163722,0.287011,0.171772
2019-01-03,0.248780,0.145758,0.252820,-0.007147,-0.586287,0.111038,0.183144,-0.284034,-0.220334,-0.662194,...,0.270851,0.131721,0.030353,0.209996,0.005119,0.282831,0.132093,0.165784,0.280088,0.159383
2019-01-04,0.278827,0.094631,0.260074,0.092268,-0.287270,0.115216,0.190969,-0.137737,-0.146667,-0.404405,...,0.278417,0.130593,0.036290,0.179895,0.005125,0.225638,0.088886,0.164170,0.283020,0.165302
2019-01-07,0.252133,0.063723,0.253737,0.031591,-0.373124,0.104516,0.174619,-0.184041,-0.107838,-0.426898,...,0.249983,0.100009,0.011376,0.224681,0.005361,0.220184,0.048170,0.142158,0.283825,0.133037
2019-01-08,0.253248,0.044006,0.253707,0.035627,-0.297597,0.102574,0.174124,-0.145075,-0.097826,-0.359614,...,0.234891,0.097884,0.011302,0.220673,0.005368,0.210700,0.045082,0.142798,0.284602,0.130012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06,-0.150961,0.477082,0.010840,-0.050779,-1.315462,0.109490,-0.487081,-0.642372,0.113115,-0.147635,...,0.139399,-0.056330,-0.080177,0.100073,0.010623,0.138200,0.141767,0.000512,0.353705,-0.080785
2024-11-07,-0.135346,0.469031,0.011553,-0.035209,-1.115661,0.106216,-0.370786,-0.574409,0.116303,-0.134182,...,0.146696,-0.052671,-0.087706,0.096005,0.010655,0.141981,0.140631,-0.002798,0.386685,-0.078476
2024-11-08,-0.156559,0.477021,0.010025,-0.069948,-1.150364,0.103122,-0.593915,-0.443186,0.122580,-0.151113,...,0.165293,-0.059310,-0.082254,0.101054,0.010687,0.156885,0.141898,-0.001429,0.423651,-0.079808


In [14]:
all_y_pred['AAPL']

Date
2019-01-02    0.071741
2019-01-03    0.057165
2019-01-04    0.062332
2019-01-07    0.085179
2019-01-08    0.089821
                ...   
2024-05-21    0.088034
2024-05-22    0.088034
2024-05-23    0.088034
2024-05-24    0.088034
2024-05-27    0.088034
Name: AAPL, Length: 1397, dtype: float64

In [9]:
# take the first row of the dataframe all_y_pred
mu = all_y_pred.iloc[0].values

In [10]:
len(mu)

746

In [11]:
from util import do_optimization, generate_features, get_doubled_matrix, get_errors_mu_short, load_pkl, load_latest_price_data, add_features, merge_fred, portfolio_log_return, portfolio_return, portfolio_volatility, portfolio_volatility_log_return, remove_nan, save_json_to_dir, update_stock_operation_and_weight


In [ ]:
# get the interest rate for the date


In [12]:
ticket_to_buy = do_optimization(mu, S, final_tickers, period, 0.01)

/home/ken/anaconda3/envs/stock/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/ken/anaconda3/envs/stock/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:438: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
/home/ken/anaconda3/envs/stock/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:492: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_eq = vstack([con['jac'](x, *con['args'])


Iteration: 1, value: -0.002961989752874719
Iteration: 2, value: -0.01965606693458305
Iteration: 3, value: -0.020488483326563126
Iteration: 4, value: -0.020617048443372532
Iteration: 5, value: -0.02063084717228554
Iteration: 6, value: -0.020634000667245397
Iteration: 7, value: -0.02063567177749652
Iteration: 8, value: -0.02063567177749652


In [ ]:
# The api-ninjas key:
api_key = "SEAZVv1FOUz54CvWYPmXjA==3c5ncx5698t467O5"

In [38]:
ticket_to_buy

[{'id': 'RHM.DE', 'weight': 0.009999999999998281},
 {'id': 'BA.L', 'weight': 0.009999999999998682},
 {'id': 'BEZ.L', 'weight': 0.009999999999999447},
 {'id': 'CCH.L', 'weight': 0.0026620611285079285},
 {'id': 'LSEG.L', 'weight': 0.009999999999999912},
 {'id': 'SMT.L', 'weight': 0.009999999999999048},
 {'id': 'ABT', 'weight': 0.007477215896061688},
 {'id': 'ABBV', 'weight': 0.009999999999998455},
 {'id': 'AMD', 'weight': 0.009999999999999749},
 {'id': 'MO', 'weight': 0.009999999999998628},
 {'id': 'AMZN', 'weight': 0.003987344691572514},
 {'id': 'AMCR', 'weight': 0.009999999999998321},
 {'id': 'AEE', 'weight': 0.009999999999998871},
 {'id': 'AWK', 'weight': 0.009999999999998857},
 {'id': 'AMGN', 'weight': 0.009999999999998795},
 {'id': 'AON', 'weight': 0.009999999999996808},
 {'id': 'APA', 'weight': 0.006138038209440245},
 {'id': 'AJG', 'weight': 0.00999999999999774},
 {'id': 'T', 'weight': 0.00999999999999806},
 {'id': 'ADP', 'weight': 0.009999999999998841},
 {'id': 'AZO', 'weight': 0.